In [2]:
# Check env vars
!env | grep -e "SPARK" -e "PYTHON"

PYSPARK_DRIVER_PYTHON=/Users/c11309a/.local/share/rtx/installs/python/3.10/bin/python
PYSPARK_PYTHON=/Users/c11309a/.local/share/rtx/installs/python/3.10/bin/python
PYTHONPATH=/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/pyspark.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip:/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3/python/lib/*.zip:
SPARK_HOME=/Users/c11309a/Tools/spark-3.3.4-bin-hadoop3
PYTHONUNBUFFERED=1
PYTHONIOENCODING=utf-8
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING=1


In [3]:
# Create a spark session
from pyspark.sql import SparkSession

spark = (
            SparkSession.builder.appName("learn_dataframes")
                .master("local[4]")
                .getOrCreate()
        )

sc = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/28 09:33:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/28 09:33:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/28 09:33:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [11]:
# Import common
from pyspark.sql.functions import udf, col, to_date
from pyspark.sql.types import StructType, StructField, DataType, StringType, IntegerType

In [21]:
# Read data from data/employees.csv and data/employees_fto.csv and join them on id
employees_df = spark.read.csv("data/employees.csv", header=True)
employees_fto_df = spark.read.csv("data/employees_fto.csv", header=True)

join_df = (
    employees_fto_df.join(employees_df, on="Id", how="left")
        .drop('salary')
        .withColumn("StartDate", to_date(col("StartDate"), "yyyy-MM-dd"))
        .withColumn("EndDate", to_date(col("EndDate"), "yyyy-MM-dd"))
)
join_df.show(truncate=False)

+---+----------+----------+--------+-----+
|Id |StartDate |EndDate   |Type    |Name |
+---+----------+----------+--------+-----+
|1  |2023-01-01|2023-01-07|Vacation|John |
|1  |2023-02-09|2023-02-11|Sick    |John |
|2  |2023-03-01|2023-03-07|Vacation|Smith|
|2  |2023-04-15|2023-04-20|Sick    |Smith|
|3  |2023-05-10|2023-05-15|Vacation|Mark |
|3  |2023-06-01|2023-06-07|Sick    |Mark |
|4  |2023-07-01|2023-07-07|Vacation|David|
|4  |2023-08-01|2023-08-07|Sick    |David|
|5  |2023-09-01|2023-09-07|Vacation|Paul |
|5  |2023-10-01|2023-10-07|Sick    |Paul |
|3  |2023-11-01|2023-11-07|Vacation|Mark |
|3  |2023-12-01|2023-12-07|Sick    |Mark |
|4  |2024-01-01|2024-01-07|Vacation|David|
|2  |2024-02-01|2024-02-07|Sick    |Smith|
|1  |2024-03-01|2024-03-07|Vacation|John |
|3  |2024-04-01|2024-04-07|Sick    |Mark |
|2  |2024-05-01|2024-05-07|Vacation|Smith|
+---+----------+----------+--------+-----+



In [22]:
# Schema
join_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- StartDate: date (nullable = true)
 |-- EndDate: date (nullable = true)
 |-- Type: string (nullable = true)
 |-- Name: string (nullable = true)



In [23]:
# Calculate days off based on start and end dates
from pyspark.sql.functions import datediff

join_df = join_df.withColumn("DaysOff", datediff(col("EndDate"), col("StartDate")))
join_df.show(truncate=False)

+---+----------+----------+--------+-----+-------+
|Id |StartDate |EndDate   |Type    |Name |DaysOff|
+---+----------+----------+--------+-----+-------+
|1  |2023-01-01|2023-01-07|Vacation|John |6      |
|1  |2023-02-09|2023-02-11|Sick    |John |2      |
|2  |2023-03-01|2023-03-07|Vacation|Smith|6      |
|2  |2023-04-15|2023-04-20|Sick    |Smith|5      |
|3  |2023-05-10|2023-05-15|Vacation|Mark |5      |
|3  |2023-06-01|2023-06-07|Sick    |Mark |6      |
|4  |2023-07-01|2023-07-07|Vacation|David|6      |
|4  |2023-08-01|2023-08-07|Sick    |David|6      |
|5  |2023-09-01|2023-09-07|Vacation|Paul |6      |
|5  |2023-10-01|2023-10-07|Sick    |Paul |6      |
|3  |2023-11-01|2023-11-07|Vacation|Mark |6      |
|3  |2023-12-01|2023-12-07|Sick    |Mark |6      |
|4  |2024-01-01|2024-01-07|Vacation|David|6      |
|2  |2024-02-01|2024-02-07|Sick    |Smith|6      |
|1  |2024-03-01|2024-03-07|Vacation|John |6      |
|3  |2024-04-01|2024-04-07|Sick    |Mark |6      |
|2  |2024-05-01|2024-05-07|Vaca

In [26]:
# Calculate total days off by Name and Type
from pyspark.sql.functions import sum, count

summary_df = (
    join_df.groupBy("Name", "Type")
        .agg(
            sum("DaysOff").alias("SumDaysOff"), 
            count("*").alias("Count")
        )
).sort("Name", "Type")

summary_df.show(truncate=False)

+-----+--------+----------+-----+
|Name |Type    |SumDaysOff|Count|
+-----+--------+----------+-----+
|David|Sick    |6         |1    |
|David|Vacation|12        |2    |
|John |Sick    |2         |1    |
|John |Vacation|12        |2    |
|Mark |Sick    |18        |3    |
|Mark |Vacation|11        |2    |
|Paul |Sick    |6         |1    |
|Paul |Vacation|6         |1    |
|Smith|Sick    |11        |2    |
|Smith|Vacation|12        |2    |
+-----+--------+----------+-----+



# Join with SQL

In [28]:
# Use SQL to join data/employees.csv and data/employees_fto.csv and calculate total days off by Name and Type
employees_df.createOrReplaceTempView("employees")
employees_fto_df.createOrReplaceTempView("employees_fto")

summary_df = spark.sql("""
    SELECT Name, Type, SUM(DATEDIFF(EndDate, StartDate)) AS SumDaysOff, COUNT(8) AS Count
    FROM employees_fto
    JOIN employees ON employees_fto.Id = employees.Id
    GROUP BY Name, Type
    ORDER BY Name, Type
""")

summary_df.show(truncate=False)

+-----+--------+----------+-----+
|Name |Type    |SumDaysOff|Count|
+-----+--------+----------+-----+
|David|Sick    |6         |1    |
|David|Vacation|12        |2    |
|John |Sick    |2         |1    |
|John |Vacation|12        |2    |
|Mark |Sick    |18        |3    |
|Mark |Vacation|11        |2    |
|Paul |Sick    |6         |1    |
|Paul |Vacation|6         |1    |
|Smith|Sick    |11        |2    |
|Smith|Vacation|12        |2    |
+-----+--------+----------+-----+

